In [1]:
#import lxml.html
#import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import time
#import math
import re
import json
import plotly.graph_objects as go
import plotly.express as px
#from bs4 import BeautifulSoup as bs4
#from selenium.webdriver import Chrome
#from selenium import webdriver
#from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

import optimize_lineup as ol
import fantasy_utils as fu

In [2]:
team_list = ['Lima Time', 'Ugly Spuds', 'Harvey', 'Roiders', 'Charmer', 'Wu Tang', '9 Grand Kids', 'Brewbirds', 'Trouble', 'Lil Trump', 'Dirty Birds', 'Young Guns'] 
eligibility_url_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [3]:
cur = pd.DataFrame({'H':[1205, 1167, 1219, 1182, 1179, 1036, 1198, 1119, 1259, 1127, 1283, 1094],
                   'AB':[4724, 4518, 4703, 4669, 4803, 4295, 4605, 4330, 4759, 4311, 4727, 4306],
                   'BA':[.255, .258, .259, .253, .245, .241, .260, .258, .265, .261, .271, .254],
                'HR':[165, 186, 205, 149, 206, 181, 177, 123, 182, 190, 186, 138],
                'R':[682, 679, 711, 588, 696, 599, 650, 584, 681, 652, 719, 557],
                'RBI':[606, 668, 698, 585, 669, 596, 648, 570, 680, 643, 652, 583],
                'SB':[108, 104, 131, 121, 134, 75, 95, 78, 107, 86, 144, 87],
                 'ERA':[3.59, 4.53, 3.81, 3.68, 3.61, 3.60, 3.94, 4.61, 3.97, 3.45, 4.09, 3.95],
                 'WHIP':[1.23, 1.25, 1.20, 1.22, 1.14, 1.16, 1.24, 1.32, 1.23, 1.15, 1.27, 1.24],
                 'W':[52, 49, 46, 55, 64, 52, 61, 44, 58, 48, 49, 53],
                 'K':[783, 911, 861, 879, 871, 823, 897, 701, 841, 766, 998, 765],
                 'Sv+Hld':[68, 73, 48, 55, 43, 35, 41, 47, 60, 56, 31, 80],
                 'IP':[785, 829.2, 767.1, 825.2, 870.2, 794.2, 857.1, 746, 818.2, 753.1, 946.2, 781.1],
                   'HA':[703, 792, 664, 755, 723, 683, 781, 744, 708, 641, 878, 711],
                   'BB':[259, 247, 260, 249, 273, 236, 285, 244, 296, 222, 322, 257],
                  }, index=sorted(team_list))
cur['ER'] = round(cur['ERA']*cur['IP']/9,0)
cur

,H,AB,BA,HR,R,RBI,SB,ERA,WHIP,W,K,Sv+Hld,IP,HA,BB,ER
9 Grand Kids,1205,4724,0.255,165,682,606,108,3.59,1.23,52,783,68,785.0,703,259,313.0
Brewbirds,1167,4518,0.258,186,679,668,104,4.53,1.25,49,911,73,829.2,792,247,417.0
Charmer,1219,4703,0.259,205,711,698,131,3.81,1.20,46,861,48,767.1,664,260,325.0
Dirty Birds,1182,4669,0.253,149,588,585,121,3.68,1.22,55,879,55,825.2,755,249,337.0
Harvey,1179,4803,0.245,206,696,669,134,3.61,1.14,64,871,43,870.2,723,273,349.0
Lil Trump,1036,4295,0.241,181,599,596,75,3.60,1.16,52,823,35,794.2,683,236,318.0
Lima Time,1198,4605,0.260,177,650,648,95,3.94,1.24,61,897,41,857.1,781,285,375.0
Roiders,1119,4330,0.258,123,584,570,78,4.61,1.32,44,701,47,746.0,744,244,382.0
Trouble,1259,4759,0.265,182,681,680,107,3.97,1.23,58,841,60,818.2,708,296,361.0
Ugly Spuds,1127,4311,0.261,190,652,643,86,3.45,1.15,48,766,56,753.1,641,222,289.0


In [4]:
def load():
    df = pd.read_csv('data\\period_18_ros_projections_.csv')
    df.loc[df['Player'].isin(['Nico Hoerner', 'Bryson Stott']), 'all_pos'] = "['2B', 'SS', 'MI', 'DH']"
    df.loc[df['Player'].isin(['Isaac Paredes', ]), 'all_pos'] = "['2B', '3B', '1B', 'MI', 'CI']"
    df.loc[df['Player'].isin(['Max Muncy', ]), 'all_pos'] = "['2B', '3B', 'MI', 'CI']"
    return df

In [5]:
def optimize_team(tm):
    w = ol.Optimized_Lineups(tm, df[df['Owner']==tm])
    print(tm)
    w._make_pitcher_combos()
    w._make_hitter_combos()
    print(w.pitcher_optimized_z, w.pitcher_optimized_lineup)
    print(w.hitter_optimized_z, w.hitter_optimized_lineup)
    return w

In [6]:
df = load()

In [7]:
df[(df['Start']==1)].groupby('Owner').apply(lambda s: pd.Series({
    'BA':round(s['H'].sum()/s['AB'].sum(),3),
    'HR':s['HR'].sum(),
    'SB':s['SB'].sum(),
    'R':s['R'].sum(),
    'RBI':s['RBI'].sum(),
    'W': s['W'].sum(),
    'SO':s['SO'].sum(),
    'Sv+Hld':s['S'].sum()+s['HD'].sum(),
    'ERA':round(s['ER'].sum()/s['IP'].sum()*9,2),
    'WHIP':round((s['Ha'].sum()+s['BBa'].sum())/s['IP'].sum(),2),
}))

,BA,HR,SB,R,RBI,W,SO,Sv+Hld,ERA,WHIP
Owner,,,,,,,,,,
9 Grand Kids,0.275,110.0,51.0,423.0,403.0,29.0,467.0,44.0,3.21,1.16
Brewbirds,0.277,130.0,66.0,458.0,447.0,32.0,531.0,39.0,3.49,1.17
Charmer,0.273,138.0,67.0,442.0,451.0,27.0,498.0,41.0,3.13,1.08
Dirty Birds,0.275,98.0,67.0,379.0,378.0,28.0,501.0,40.0,3.35,1.15
Harvey,0.267,116.0,67.0,417.0,407.0,36.0,528.0,14.0,3.61,1.18
Lil Trump,0.261,122.0,43.0,407.0,405.0,31.0,510.0,14.0,3.86,1.19
Lima Time,0.275,120.0,59.0,410.0,421.0,40.0,579.0,21.0,3.61,1.20
Roiders,0.283,107.0,47.0,414.0,399.0,36.0,517.0,28.0,3.43,1.16
Trouble,0.277,113.0,77.0,425.0,410.0,32.0,522.0,28.0,3.47,1.18


In [8]:
#c = ol.Optimized_Lineups('Charmer', df[df['Owner']=='Charmer'])
#h = ol.Optimized_Lineups('Harvey', df[df['Owner']=='Harvey'])
w = ol.Optimized_Lineups('Wu Tang', df[df['Owner']=='Wu Tang'])
l = ol.Optimized_Lineups('Lima Time', df[df['Owner']=='Lima Time'])
#t = ol.Optimized_Lineups('Trouble', df[df['Owner']=='Trouble'])
u = ol.Optimized_Lineups('Ugly Spuds', df[df['Owner']=='Ugly Spuds'])

In [9]:
tm = 'Lima Time'
l = optimize_team(tm)

Lima Time
190120
190120
1.462902428225196 ('Yu Darvish', 'Shane McClanahan', 'Charlie Morton', 'Justin Steele', 'Cristian Javier', 'Lucas Giolito', 'Hunter Brown', 'Jordan Montgomery', 'Jason Adam')
0.7952436587042151 ['Adley Rutschman', 'Jake Burger', 'Ryan McMahon', 'Xander Bogaerts', 'Max Muncy', 'Corey Seager', 'Josh Naylor', 'Kyle Tucker', 'Alex Verdugo', 'Jarren Duran', 'TJ Friedl', 'Jorge Soler', 'Jose Altuve', 'Alex Bregman']


In [11]:
df[df['Player'].isin(l.pitcher_optimized_lineup)]['ER'].sum() / df[df['Player'].isin(l.pitcher_optimized_lineup)]['IP'].sum() * 9

3.4976427255985265

In [10]:
rps = ['Peter Fairbanks', 'Tanner Scott', 'Jason Adam', 'Matt Brash', 'Eric Swanson', 'Adbert Alzolay', 'Jhoan Duran', 'Jordan Montgomery', 'Justin Steele']
df[df['Player'].isin(rps)]['ER'].sum() / df[df['Player'].isin(rps)]['IP'].sum() * 9

3.0535294117647065

In [12]:
c = optimize_team('Charmer')

Charmer
62720
62720
10.666806928701355 ('Shohei Ohtani', 'Evan Phillips', 'Kodai Senga', 'Joe Ryan', 'Eduardo Rodriguez', 'Jhoan Duran', 'Liam Hendriks', 'Matt Manning', 'Bryan Abreu')
4.7796575233792495 ['Shea Langeliers', 'Pete Alonso', 'Ketel Marte', 'Mookie Betts', 'Jose Ramirez', 'Brandon Lowe', 'Ryan Mountcastle', 'Bryan Reynolds', 'Giancarlo Stanton', 'Seiya Suzuki', 'Spencer Steer', 'Esteury Ruiz', 'Matt McLain', 'Ha-seong Kim']


In [18]:
h = optimize_team('Harvey')

Harvey
17370
17370
-2.419429529999999 ('Sandy Alcantara', 'George Kirby', 'Edward Cabrera', 'Emmet Sheehan', 'Will Smith', 'Tarik Skubal', 'Jon Gray', 'Nestor Cortes', 'Joel Payamps')
-4.239603088999998 ['William Contreras', 'Jose Abreu', 'Jonathan India', 'Anthony Volpe', 'Alec Bohm', 'Amed Rosario', 'Rafael Devers', 'Julio Rodriguez', 'Fernando Tatis', 'Corbin Carroll', 'Kyle Schwarber', 'James Outman', 'Marcell Ozuna', 'Garrett Cooper']


In [12]:
w = optimize_team('Wu Tang')

Wu Tang
152880
152880
0.41394528484637494 ('Spencer Strider', 'Tony Gonsolin', 'Justin Verlander', 'Peter Fairbanks', 'Chris Bassitt', 'Shane Bieber', 'Johan Oviedo', 'Kyle Finnegan', 'Adam Ottavino')
14.346891677304711 ['Francisco Alvarez', 'Cody Bellinger', 'Justin Turner', 'Nico Hoerner', 'Manny Machado', 'Javier Baez', 'Nate Lowe', 'Ronald Acuna', 'Luis Robert', 'Jazz Chisholm', 'Mike Trout', 'Mickey Moniak', 'Masataka Yoshida', 'Nolan Jones']


In [13]:
u = optimize_team('Ugly Spuds')

Ugly Spuds
0
0


ValueError: attempt to get argmax of an empty sequence

In [19]:
#df2 = load()
df.loc[df['Player'].isin(['Adbert Alzolay', 'Peter Fairbanks', 'Nico Hoerner']), 'Owner'] = 'Lima Time'
df.loc[df['Player'].isin(['Yu Darvish', 'Ji-HWan Bae']), 'Owner'] = 'Wu Tang'


In [15]:
df2[df2['Owner']=='Lima Time']['Sv+Hld'].sum()

58.0

In [16]:
print(f"ROS ERA would be {round(df[df['Player'].isin(rps)]['ER'].sum() / df[df['Player'].isin(rps)]['IP'].sum() * 9,2)}")
print(f"ROS WHIP would be {round((df[df['Player'].isin(rps)]['BBa'].sum() + df[df['Player'].isin(rps)]['Ha'].sum()) / df[df['Player'].isin(rps)]['IP'].sum(),2)}")
print(f"ROS W would be {df[df['Player'].isin(rps)]['W'].sum()}")

ROS ERA would be 3.05
ROS WHIP would be 1.16
ROS W would be 13.0


In [21]:
print(f"ERA would be {(cur.loc['Lima Time']['ER'] + df[df['Player'].isin(rps)]['ER'].sum()) / (cur.loc['Lima Time']['IP'] + df[df['Player'].isin(rps)]['IP'].sum()) * 9}")
print(f"WHIP would be {(cur.loc['Lima Time']['BB'] + cur.loc['Lima Time']['HA'] + df[df['Player'].isin(rps)]['BBa'].sum() + df[df['Player'].isin(rps)]['Ha'].sum()) / (cur.loc['Lima Time']['IP'] + df[df['Player'].isin(rps)]['IP'].sum())}")
print(f"W would be {cur.loc['Lima Time']['W'] + df[df['Player'].isin(rps)]['W'].sum()}")
print(f"K would be {cur.loc['Lima Time']['K'] + df[df['Player'].isin(rps)]['K'].sum()}")
print(f"Sv+Hld would be {cur.loc['Lima Time']['Sv+Hld'] + df[df['Player'].isin(rps)]['Sv+Hld'].sum()}")

ERA would be 3.7564511640849645
WHIP would be 1.2262313329004733
W would be 74.0
K would be 897.0
Sv+Hld would be 85.0


In [109]:
proj = cur.merge(df.groupby('Owner')['Sv+Hld'].sum(), left_index=True, right_index=True)
proj['Sv+Hld'] = proj['Sv+Hld_x']+proj['Sv+Hld_y']
proj.sort_values('Sv+Hld', ascending=False)

,H,AB,BA,HR,R,RBI,SB,ERA,WHIP,W,K,Sv+Hld_x,IP,HA,BB,ER,Sv+Hld_y,Sv+Hld
Young Guns,1094,4306,0.254,138,557,583,87,3.95,1.24,53,765,80,781.1,711,257,343.0,63,143
Brewbirds,1167,4518,0.258,186,679,668,104,4.53,1.25,49,911,73,829.2,792,247,417.0,67,140
Trouble,1259,4759,0.265,182,681,680,107,3.97,1.23,58,841,60,818.2,708,296,361.0,56,116
9 Grand Kids,1205,4724,0.255,165,682,606,108,3.59,1.23,52,783,68,785.0,703,259,313.0,47,115
Lima Time,1198,4605,0.260,177,650,648,95,3.94,1.24,61,897,41,857.1,781,285,375.0,72,113
Ugly Spuds,1127,4311,0.261,190,652,643,86,3.45,1.15,48,766,56,753.1,641,222,289.0,51,107
Dirty Birds,1182,4669,0.253,149,588,585,121,3.68,1.22,55,879,55,825.2,755,249,337.0,47,102
Charmer,1219,4703,0.259,205,711,698,131,3.81,1.20,46,861,48,767.1,664,260,325.0,50,98
Roiders,1119,4330,0.258,123,584,570,78,4.61,1.32,44,701,47,746.0,744,244,382.0,37,84
Harvey,1179,4803,0.245,206,696,669,134,3.61,1.14,64,871,43,870.2,723,273,349.0,38,81


In [108]:
df[df['Player'].isin(rps)]['Sv+Hld'].sum()

86

In [20]:
l2 = optimize_team('Lima Time')
print('\n')
c2 = optimize_team('Wu Tang')

Lima Time
611808
611808
0.38238982172212344 ('Shane McClanahan', 'Charlie Morton', 'Justin Steele', 'Cristian Javier', 'Lucas Giolito', 'Hunter Brown', 'Jordan Montgomery', 'Jason Adam', 'Peter Fairbanks')
3.0814588390737883 ['Adley Rutschman', 'Josh Naylor', 'Jose Altuve', 'Corey Seager', 'Ryan McMahon', 'Max Muncy', 'Alex Bregman', 'Kyle Tucker', 'Alex Verdugo', 'Jarren Duran', 'TJ Friedl', 'Jorge Soler', 'Xander Bogaerts', 'Nico Hoerner']


Wu Tang
9576
9576
1.4944578913494473 ('Yu Darvish', 'Spencer Strider', 'Tony Gonsolin', 'Justin Verlander', 'Chris Bassitt', 'Shane Bieber', 'Johan Oviedo', 'Kyle Finnegan', 'Adam Ottavino')
12.722854036057534 ['Francisco Alvarez', 'Cody Bellinger', 'Justin Turner', 'Javier Baez', 'Manny Machado', 'Bryson Stott', 'Nate Lowe', 'Ronald Acuna', 'Luis Robert', 'Jazz Chisholm', 'Mike Trout', 'Nolan Jones', 'Masataka Yoshida', 'Mickey Moniak']


In [15]:
l3 = optimize_team('Lima Time')
print('\n')
w3 = optimize_team('Ugly Spuds')

Lima Time
2520
2520
11.193824977542931 ('Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Erik Swanson', 'Jason Adam', 'Yu Darvish', 'Sandy Alcantara')
-11.070364937530119 ['Adley Rutschman', 'Josh Naylor', 'Max Muncy', 'Xander Bogaerts', 'Jake Burger', 'Ji-Hwan Bae', 'Vinnie Pasquantino', 'Kyle Tucker', 'Alex Verdugo', 'Andrew Benintendi', 'TJ Friedl', 'Trevor Larnach', 'Jose Altuve', 'Alex Bregman']


Ugly Spuds
141120
141120
1.8660023719271694 ('Justin Verlander', 'Framber Valdez', 'Luis Castillo', 'Drew Rasmussen', 'Brandon Woodruff', 'Tyler Mahle', 'Giovanny Gallegos', 'Jorge Lopez', 'James Karinchak')
-9.581041050785146 ['Will Smith', 'Alec Bohm', 'Isaac Paredes', 'Corey Seager', 'Josh Jung', 'Bo Bichette', 'C.J. Cron', 'Nick Castellanos', 'Mike Yastrzemski', 'Christian Yelich', 'Jack Suwinski', 'Oscar Gonzalez', 'Harrison Bader', 'Harold Ramirez']


In [57]:
#df = load()
df.loc[df['Player'].isin(['Jordan Montgomery']), 'Owner'] = 'Ugly Spuds'
df.loc[df['Player'].isin(['Yennier Cano']), 'Owner'] = 'Lima Time'

In [15]:
l4 = optimize_team('Lima Time')
print('\n')
h4 = optimize_team('Harvey')

Lima Time
598192
598192
5.520648398669712 ('Justin Verlander', 'Shane McClanahan', 'Cristian Javier', 'Lucas Giolito', 'Charlie Morton', 'Justin Steele', 'Jordan Montgomery', 'Erik Swanson', 'Jason Adam')
-2.8852745859699453 ['Adley Rutschman', 'Vinnie Pasquantino', 'Max Muncy', 'Xander Bogaerts', 'Ryan McMahon', 'Ji-Hwan Bae', 'Alec Bohm', 'James Outman', 'Kyle Tucker', "Tyler O'Neill", 'Alex Verdugo', 'Andrew Benintendi', 'Jose Altuve', 'Josh Naylor']


Harvey
2328
2328
2.689226586203938 ('Dustin May', 'Tony Gonsolin', 'Nestor Cortes', 'Edward Cabrera', 'Jon Gray', 'Kyle Wright', 'George Kirby', 'Anthony DeSclafani', 'Alex Lange')
6.998851462626627 ['William Contreras', 'Jose Abreu', 'Jonathan India', 'Anthony Volpe', 'Rafael Devers', 'Amed Rosario', 'Triston Casas', 'Fernando Tatis', 'Jazz Chisholm', 'Kyle Schwarber', 'Corbin Carroll', 'Brian Anderson', 'Julio Rodriguez', 'Patrick Wisdom']


In [ ]:
df[['Player', 'BA', 'HR', 'SB', 'R', 'RBI', 'z']].sort_values('z', ascending=False)

In [91]:
df[df['Player'].isin(['Justin Verlander', 'Alec Bohm', 'Alex Bregman', 'Yu Darvish'])] .groupby(['Owner', 'Player'])[['HR', 'SB', 'R', 'RBI', 'BA', 'z', 'IP', 'ERA', 
                                                                         'WHIP', 'W', 'SO', 'Sv+Hld']].sum()

HR   SB     R   RBI     BA         z     IP  \
Owner      Player                                                            
Lima Time  Alex Bregman      19.0  1.0  78.0  73.0  0.271  0.694379    0.0   
           Yu Darvish         0.0  0.0   0.0   0.0  0.000  2.272376  157.0   
Ugly Spuds Alec Bohm         11.0  2.0  52.0  67.0  0.282 -2.180454    0.0   
           Justin Verlander   0.0  0.0   0.0   0.0  0.000  3.389474  151.0   

                              ERA  WHIP     W     SO  Sv+Hld  
Owner      Player                                             
Lima Time  Alex Bregman      0.00  0.00   0.0    0.0     0.0  
           Yu Darvish        3.21  1.11  11.0  163.0     0.0  
Ugly Spuds Alec Bohm         0.00  0.00   0.0    0.0     0.0  
           Justin Verlander  2.80  1.16  14.0  156.0     0.0

In [ ]:
df.loc[df['Player']=='Grayson Rodriguez', ['W', 'SO', 'ERA', 'WHIP']] = [7, 131, 3.8, 1.2]

In [ ]:
df[df['Player'].isin(w.pitcher_optimized_lineup+l.pitcher_optimized_lineup)].groupby('Owner').apply(lambda s: pd.Series({
    'W': s['W'].sum(),
    'SO':s['SO'].sum(),
    'S':s['S'].sum()+s['HD'].sum(),
    'ERA':round(s['ER'].sum()/s['IP'].sum()*9,2),
    'WHIP':round((s['Ha'].sum()+s['BBa'].sum())/s['IP'].sum(),2),
}))

In [ ]:
df[df['Player'].isin(['Jose Abreu', 'Vinnie Pasquantino', 'Justin Steele', 'Brian Anderson', 'Amed Rosario', 
                      'Reynaldo Lopez'])] .groupby(['Owner', 'Player'])[['AB','HR', 'SB', 'R', 'RBI', 'BA', 'z', 'IP', 'ERA', 
                                                                         'WHIP', 'W', 'SO', 'Sv+Hld']].sum()

In [ ]:
df[df['Player'].isin(['Jose Abreu', 'Vinnie Pasquantino', 'Justin Steele', 'Brian Anderson', 'Amed Rosario', 
                      'Reynaldo Lopez'])] .groupby('Owner')[['AB','HR', 'SB', 'R', 'RBI', 'BA', 'z', 'IP', 'ERA', 
                                                                         'WHIP', 'W', 'SO', 'Sv+Hld']].sum()

In [ ]:
h.pitcher_optimized_lineup, l.pitcher_optimized_lineup

In [ ]:
print(f"Harvey\nhitting: {h.hitter_optimized_z}\npitching: {h.pitcher_optimized_z}")
#print(f"Lima\nhitting: {l.hitter_optimized_z}\npitching: {l.pitcher_optimized_z}")

In [84]:
df = load()
df.loc[df['Player'].isin(['Jhoan Duran']), 'Owner'] = 'Lima Time'
df.loc[df['Player'].isin(['Lucas Giolito']), 'Owner'] = 'Charmer'
df.loc[df['Player'].isin(['Alexis Diaz', 'Andres Munoz']), 'Owner'] = 'Lima Time'
df.loc[df['Player'].isin(['Yu Darvish', 'Scott McGough']), 'Owner'] = 'Brewbirds'

In [75]:
df[df['Player'].isin(['Tanner Scott', 'Matt Strahm', 'Ian Gibaut', 'Alexis Diaz', 'Andres Munoz', 'Jhoan Duran', 'Scott McGough', 'Erik Swanson'])]

,Unnamed: 0,index,Avail,Player,AB,R,H,HR,RBI,BB,K,SB,CS,BA,OBP,SLG,Rank,Team,Pos,Owner,top_50,top_100,top_276,above_276,Start,IP,APP,GS,QS,CG,W,L,S,BS,HD,SO,BBa,Ha,ERA,WHIP,Sv+Hld,ER,C,1B,2B,3B,SS,MI,CI,OF,DH,SP,RP,P,type,zlgBA,zlgERA,zlgWHIP,zR,zHR,zRBI,zSB,zzlgBA,BIGAAh,zW,zSO,zSv+Hld,zzlgERA,zzlgWHIP,BIGAAp,z,all_pos
56,56,4,Brewbi...,Alexis Diaz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0,CIN,RP,Lima Time,0.0,0.0,1.0,0.0,1.0,23.0,26.0,0.0,0.0,0.0,1.0,1.0,13.0,1.0,1.0,38.0,12.0,14.0,2.35,1.13,14.0,6.005556,0,0,0,0,0,0,0,0,0,0,46,5.0,p,NaN,25.202330,0.896805,NaN,NaN,NaN,NaN,NaN,0.0,-1.534964,-1.118204,1.710174,0.970349,0.173474,0.200829,0.200829,"['RP', 'P']"
63,63,11,Brewbi...,Andres Munoz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,489.0,SEA,RP,Lima Time,0.0,0.0,0.0,1.0,0.0,24.0,26.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,11.0,35.0,8.0,16.0,2.62,1.00,11.0,6.986667,0,0,0,0,0,0,0,0,0,0,21,5.0,p,NaN,19.818083,4.066232,NaN,NaN,NaN,NaN,NaN,0.0,-1.534964,-1.293157,1.225995,0.763043,0.786553,-0.052531,-0.052531,"['RP', 'P']"
88,89,5,Charme...,Jhoan Duran,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,MIN,RP,Lima Time,0.0,0.0,1.0,0.0,1.0,25.0,25.0,0.0,0.0,0.0,1.0,1.0,11.0,2.0,1.0,35.0,8.0,18.0,2.16,1.04,12.0,6.000000,0,0,0,0,0,0,0,0,0,0,37,5.0,p,NaN,32.143837,3.235658,NaN,NaN,NaN,NaN,NaN,0.0,-1.534964,-1.293157,1.387388,1.237613,0.625890,0.422771,0.422771,"['RP', 'P']"
224,226,8,Lima T...,Scott McGough,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,419.0,ARI,RP,Brewbirds,0.0,0.0,0.0,1.0,1.0,29.0,28.0,0.0,0.0,0.0,0.0,3.0,11.0,2.0,10.0,43.0,14.0,25.0,4.66,1.34,21.0,15.015556,0,0,0,0,0,0,0,0,0,0,43,5.0,p,NaN,-35.213149,-5.086637,NaN,NaN,NaN,NaN,NaN,0.0,-2.119967,-0.826615,2.839925,-1.355789,-0.983935,-2.446381,-2.446381,"['RP', 'P']"
227,229,11,Lima T...,Matt Strahm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,569.0,PHI,RP,Lima Time,0.0,0.0,0.0,1.0,0.0,21.0,24.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,27.0,6.0,18.0,3.86,1.14,4.0,9.006667,0,0,0,0,0,0,0,0,0,9,22,5.0,p,NaN,-8.699177,0.557953,NaN,NaN,NaN,NaN,NaN,0.0,-0.949960,-1.759699,0.096244,-0.334939,0.107928,-2.840427,-2.840427,"['SP', 'RP', 'P']"
228,230,12,Lima T...,Tanner Scott,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,611.0,MIA,RP,Lima Time,0.0,0.0,0.0,1.0,0.0,23.0,28.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,9.0,35.0,12.0,19.0,3.52,1.35,10.0,8.995556,0,0,0,0,0,0,0,0,0,0,47,5.0,p,NaN,-1.707670,-4.103195,NaN,NaN,NaN,NaN,NaN,0.0,-1.534964,-1.293157,1.064602,-0.065749,-0.793703,-2.622971,-2.622971,"['RP', 'P']"
229,231,13,Lima T...,Erik Swanson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,621.0,TOR,RP,Lima Time,0.0,0.0,0.0,1.0,0.0,17.0,9.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,20.0,5.0,12.0,2.65,1.00,1.0,5.005556,0,0,0,0,0,0,0,0,0,0,47,5.0,p,NaN,13.527809,2.880248,NaN,NaN,NaN,NaN,NaN,0.0,-1.534964,-2.167924,-0.387936,0.520852,0.557142,-3.012829,-3.012829,"['RP', 'P']"


In [95]:
df.loc[229, ['IP', 'Sv+Hld', 'W', 'SO', 'Ha', 'BBa', 'ER', 'ERA', 'WHIP']] = [24, 11, 1, 24, 24, 0, 7, 2.65, 1.00]
#df.loc[]

In [97]:
rps = ['Jhoan Duran', 'Alexis Diaz', 'Justin Steele', 'Cristian Javier', 'Shane McClanahan', 'Andres Munoz', 'Erik Swanson', 'Charlie Morton', 'Jason Adam']
#rps = ['Alexis Diaz', 'Andres Munoz', 'Jhoan Duran', 'Shane McClanahan', 'Charlie Morton', 'Justin Steele', 'Cristian Javier', 'Hunter Brown', 'Jason Adam']
#rps = ['Jhoan Duran', 'Yu Darvish', 'Shane McClanahan', 'Charlie Morton', 'Justin Steele', 'Cristian Javier', 'Hunter Brown', 'Jordan Montgomery', 'Jason Adam']

In [98]:
ol_dict = {}
for t in df.Owner.sort_values().unique():
    print(t)
    ol_dict[t] = ol.Optimized_Lineups(t, df[df['Owner']==t])
    ol_dict[t]._make_pitcher_combos()
    #ol_dict[t]._make_hitter_combos()
    #print(f'Offense: {ol_dict[t].hitter_optimized_z}')
    print(f'Pitching: {ol_dict[t].pitcher_optimized_z}')
    if t=='Lima Time':
        print(ol_dict[t].pitcher_optimized_lineup)
        print(f"Sv+Hld {df[df['Player'].isin(rps)]['Sv+Hld'].sum()}")
        print(f"{df[df['Player'].isin(rps)][['IP','Ha','BBa', 'ER', 'W','SO','Sv+Hld']].sum()}")
    else:
        print(ol_dict[t].pitcher_optimized_lineup)
        print(f"Sv+Hld: {df[df['Player'].isin(ol_dict[t].pitcher_optimized_lineup)]['Sv+Hld'].sum()}")
        print(f"{df[df['Player'].isin(ol_dict[t].pitcher_optimized_lineup)][['IP','Ha','BBa', 'ER', 'W','SO','Sv+Hld']].sum()}")
    #print(f'Total z: {ol_dict[t].pitcher_optimized_z + ol_dict[t].hitter_optimized_z}')

9 Grand Kids
Pitching: 4.270081106824831
('Zac Gallen', 'Josh Hader', 'Devin Williams', 'Clayton Kershaw', 'Aaron Civale', 'Domingo German', 'Kenley Jansen', 'Bryce Elder', 'JP Sears')
Sv+Hld: 44.0
IP        453.000000
Ha        376.000000
BBa       132.000000
ER        159.976667
W          27.000000
SO        455.000000
Sv+Hld     44.000000
dtype: float64
Brewbirds
Pitching: 2.847832566548882
('Max Scherzer', 'Aaron Nola', 'Taj Bradley', 'Logan Webb', 'Lance Lynn', 'Craig Kimbrel', 'Carlos Estevez', 'Julio Teheran', 'Yu Darvish')
Sv+Hld: 25.0
IP        517.000000
Ha        454.000000
BBa       141.000000
ER        204.972222
W          36.000000
SO        555.000000
Sv+Hld     25.000000
dtype: float64
Charmer
Pitching: 10.127999675384373
('Shohei Ohtani', 'Evan Phillips', 'Kodai Senga', 'Joe Ryan', 'Eduardo Rodriguez', 'Liam Hendriks', 'Matt Manning', 'Bryan Abreu', 'Lucas Giolito')
Sv+Hld: 38.0
IP        455.000000
Ha        363.000000
BBa       134.000000
ER        160.953333
W    

In [86]:
ol_dict['Lima Time'].pitcher_optimized_lineup

('Alexis Diaz',
 'Andres Munoz',
 'Jhoan Duran',
 'Shane McClanahan',
 'Charlie Morton',
 'Justin Steele',
 'Cristian Javier',
 'Hunter Brown',
 'Jason Adam')

In [ ]:
for k, v in ol_dict['Brewbirds'].h_dict.items():
    print(k,v['all_pos'])
    if "OF" in v['all_pos']:#.replace('[','').replace(']','').replace('\'','').split(", "):
        print('yes')

In [ ]:
def get_team_projections(owner):
    b = pd.read_html(str(soup.find_all('table')[1]), header=1, converters={'AB':str_2_int,'H':str_2_int, 'R':str_2_int,'HR':str_2_int,'RBI':str_2_int,'SB':str_2_int})[0].drop(columns='Edit')
    b = b[b['Pos']!='Reserves'].reset_index(drop=False).drop(columns='index')
    b.loc[b.index<14, 'Status'] = 'Start'
    b.loc[b.index>=14, 'Status'] = 'Bench'
    b['Name'] = b.Players.apply(lambda x: parse_name(x))
    b['Elig'] = b.Players.apply(lambda x: parse_pos(x))
    b['Team'] = b.Players.apply(lambda x: parse_team(x))
    b['Owner'] = owner
    
    p = pd.read_html(str(soup.find_all('table')[2]), header=1, converters={'INNs':str_2_int,'BBI':str_2_int, 'HA':str_2_int, 'W':str_2_int,'S':str_2_int,'K':str_2_int})[0].drop(columns='Edit')
    p = p[~p['Pos'].str.contains('Reserve')].reset_index(drop=False).drop(columns='index')
    p['ER'] = (p['ERA'].astype(float)/9)*p['INNs'].astype(float)
    p.loc[p.index<9, 'Status'] = 'Start'
    p.loc[p.index>=9, 'Status'] = 'Bench'
    p['Name'] = p.Players.apply(lambda x: parse_name(x))
    p['Elig'] = p.Players.apply(lambda x: parse_pos(x))
    p['Team'] = p.Players.apply(lambda x: parse_team(x))
    p['Owner'] = owner
    return b, p

In [ ]:
#df = b[b['Status']=='Start'].groupby('Owner').agg({'R':'sum', 'RBI':'sum', 'HR':'sum', 'SB':'sum', 'H':'sum', 'AB':'sum'})
def get_proj_batting_stats_by_owner(b):
    df = b[b['Status']=='Start'].groupby("Owner").apply(lambda s: pd.Series({ 
        "HR": s["HR"].sum(), 
        "R": s["R"].sum(),
        'RBI': s['RBI'].sum(),
        'SB': s['SB'].sum(),
        "BA": s["H"].sum()/s["AB"].sum(),
        'H': s['H'].sum(),
        'AB': s['AB'].sum()
    }))
    return df

def get_batting_ranks(proj):
    b_ranks = proj.drop(columns=['H', 'AB']).rank()
    b_ranks['Pts'] = b_ranks.sum(axis=1)
    return b_ranks

proj_batting_stats_by_owner = get_proj_batting_stats_by_owner(b)
b_ranks = get_batting_ranks(proj_batting_stats_by_owner)

In [ ]:
dfp = p[p['Status']=='Start'].groupby("Owner").apply(lambda s: pd.Series({ 
    "W": s["W"].sum(), 
    "K": s["K"].sum(),
    'S': s['S'].sum(),
    'ERA': s['ER'].sum()/s['INNs'].sum()*9,
    "WHIP": (s["HA"].sum()+s['BBI'].sum())/s["INNs"].sum(),
}))

p_ranks = dfp.rank()
p_ranks['Pts'] = p_ranks.sum(axis=1)
p_ranks.sort_values('Pts', ascending=False)

In [ ]:
dfp.sort_values('W', ascending=False)

In [ ]:
(b_ranks['Pts']+p_ranks['Pts']).sort_values().plot(kind='bar')

In [ ]:
z_table = b[b['Status']=='Start'].agg({'HR':['mean','std'], 'R':['mean', 'std'], 'RBI':['mean','std'], 'SB':['mean','std'], 'H':['mean','std'], 'AB':['mean','std']})
z_table = z_table.join(p[p['Status']=='Start'].agg({'W':['mean','std'], 'K':['mean', 'std'], 'S':['mean','std'], 'ER':['mean','std'], 'HA':['mean','std'], 'BBI':['mean','std'], 'INNs':['mean', 'std']}))
z_table

In [ ]:
for stat in ['R', 'HR', 'RBI', 'SB']:
    b['z'+stat] = (b[stat]-z_table[stat]['mean']) / z_table[stat]['std']

b['zBA'] = b['H']-(b['AB']*(z_table['H']['mean']) / z_table['AB']['mean'])
b['zBA'] = (b['zBA']-b['zBA'].mean())/b['zBA'].std()
b['z'] = b['zBA'] + b['zHR'] + b['zR'] + b['zRBI'] + b['zSB']

for stat in ['W', 'K', 'S']:
    p['z'+stat] = (p[stat]-z_table[stat]['mean']) / z_table[stat]['std']

p['zER'] = ((p['ER']*9) - ((p['INNs'] * z_table['ER']['mean'] * 9) / z_table['INNs']['mean'])) * -1
p['zERA'] = (p['zER'] - p['zER'].mean()) / p['zER'].std()
p['zH+BB'] = ((p['HA']+p['BBI']) - (p['INNs'] * ((z_table['HA']['mean'] + z_table['BBI']['mean']) / z_table['INNs']['mean']))) * -1
p['zWHIP'] = (p['zH+BB'] - p['zH+BB'].mean()) / p['zH+BB'].std()
p['z'] = p['zERA'] + p['zWHIP'] + p['zW'] + p['zS'] + p['zK']

In [ ]:
p['zER'].mean(), p['zER'].std()
p.sort_values('z').tail(15)

In [ ]:
conv = (260*12)/(b[b['z']>0].z.sum() + p[p['z']>0].z.sum())
conv

In [ ]:
b['$'] = b['z'] * conv *.65
p['$'] = p['z'] * conv *.35

In [ ]:
b[b['Status']=='Start'].groupby('Owner')['$'].sum()

In [ ]:
#ba_pts = row['H']-(row['AB']*(qual_avgs['H'][0]/qual_avgs['AB'][0]))
#zBA = (ba_pts-qual_avgs['zlgBA'][0])/qual_avgs['zlgBA'][1]
play = b[(b['Owner']=='Lima Time') & (b['Status']=='Start')]
bench = b[(b['Owner']=='Lima Time') & (b['Status']=='Bench')]

In [ ]:
play

In [ ]:
bench

In [ ]:
def get_position_list(player):
    elig = player.split(',')
    for pos in elig:
        if pos in ['1B', '3B']:
            elig.extend(['CI'])
        elif pos in ['2B', 'SS']:
            elig.extend(['MI'])
    elig.extend(['DH'])
    return list(dict.fromkeys(elig))

def check_if_better(cur, new):
    if new > cur:
        return new
    else:
        return cur

In [ ]:
# Line-up Optimization Algorithm
base_z = play['z'].sum()

for bench_spot in range(bench.shape[0]):
    optimal_z = play['z'].sum()

    for i in get_position_list(bench['Elig'].iloc[bench_spot]):
        mask = (play['Pos']==i)
        if i in ['C', '1B', '2B', 'SS', '3B', 'MI', 'CI']:
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[0], optimal_z)
        if i=='OF':
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[0], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[1], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[2], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[3], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[4], optimal_z)
        if i=='DH':
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[0], optimal_z)
            optimal_z = check_if_better(base_z + bench.iloc[bench_spot]['z'] - play[mask]['z'].iloc[1], optimal_z)
    
    if optimal_z > play['z'].sum():
        print('The line-up is not optimal.',bench.iloc[bench_spot]['Name'],'should go in.')
    else:
        print('The line-up is already optimal.',bench.iloc[bench_spot]['Name'],'should stay on the bench.')

In [ ]:
b[b['Elig'].str.contains('OF')].groupby('Owner')['z'].mean()

In [ ]:
b[b['Owner']=='Brewbirds']

In [ ]:
trade = b.copy()

In [ ]:
trade.loc[trade['Name']=='Nolan Arenado', 'Owner'] = 'Lima Time'
trade.loc[trade['Name']=='Tyler O\'Neill', 'Owner'] = 'Dirty Birds'

In [ ]:
trade[trade['Name'].isin(['Ryan McMahon', 'Tyler O\'Neill'])]

In [ ]:
get_proj_batting_stats_by_owner(trade)

In [ ]:
proj_batting_stats_by_owner

In [ ]:
get_batting_ranks(get_proj_batting_stats_by_owner(trade)).sort_values('Pts', ascending=False)

In [ ]:
from pybaseball.analysis.projections.marcels import MarcelProjectionsPitching
from pybaseball.analysis.projections.marcels import MarcelProjectionsBatting
from pybaseball.lahman import people

In [ ]:
ppl = people()  # for merging player names onto the projections
ppl['Player'] = ppl['nameFirst'].str.cat(ppl['nameLast'], ' ')
marcel_batting = MarcelProjectionsBatting()
marcel_pitching = MarcelProjectionsPitching()

In [ ]:
pitchers = marcel_pitching.projections(2022)
batters = marcel_batting.projections(2022)
pitchers = pd.merge(ppl[['playerID', 'Player']], pitchers, on='playerID', how='right')
batters = pd.merge(ppl[['playerID', 'Player']], batters, on='playerID', how='right')

In [ ]:
pitchers.sort_values('SO', ascending=False).head()

In [ ]:
batters.sort_values('HR', ascending=False).head(20)